In [2]:
import tensorflow as tf

In [3]:
import os
import shutil
import random

raw_dir=r'D:\Dataset\mrlEyes_2018_01'
file_path=[]
for dirpath, dirname, filenames in os.walk(raw_dir):
   for i  in [f for f in filenames if f.endswith('.png')]:
       file_path.append(os.path.join(dirpath,i))
random.shuffle(file_path)


In [4]:
train_len=int(0.8*len(file_path))
test_len=len(file_path)-train_len

In [5]:
count=0

for i in file_path:
    file_name = os.path.basename(i)
    if count<train_len:
        if file_name.split('_')[4]=='0':
            shutil.copy(src=i, dst=r'D:\Dataset\mrlEyes_prepared_data\train_data\closed_eyes')
        elif file_name.split('_')[4]=='1':
            shutil.copy(src=i, dst=r'D:\Dataset\mrlEyes_prepared_data\train_data\open_eyes')
    else:
        if file_name.split('_')[4]=='0':
            shutil.copy(src=i, dst=r'D:\Dataset\mrlEyes_prepared_data\test_data\closed_eyes')
        elif file_name.split('_')[4]=='1':
            shutil.copy(src=i, dst=r'D:\Dataset\mrlEyes_prepared_data\test_data\open_eyes')
    count+=1
        

In [6]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
train_datagen=ImageDataGenerator(rescale=1./255,rotation_range=0.2,
                              shear_range=0.2,
                              zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,validation_split=0.2)
train_data=train_datagen.flow_from_directory(r'D:\Dataset\mrlEyes_prepared_data\train_data',target_size=(80,80),batch_size=8,class_mode='categorical',subset='training')
validation_data=train_datagen.flow_from_directory(r'D:\Dataset\mrlEyes_prepared_data\train_data',target_size=(80,80),batch_size=8,class_mode='categorical',
                                                 subset='validation')

Found 65109 images belonging to 2 classes.
Found 16276 images belonging to 2 classes.


In [8]:
test_datagen=ImageDataGenerator(rescale=1./255)

test_data=test_datagen.flow_from_directory(r'D:\Dataset\mrlEyes_prepared_data\test_data',target_size=(80,80),batch_size=8,class_mode='categorical')

Found 30447 images belonging to 2 classes.


In [9]:
base_model= InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
head_model=base_model.output

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint=ModelCheckpoint(r'D:\Model\DD_Project\model checkpoint\model.keras',monitor='val_loss',save_best_only=True,verbose=3)

earlystop=EarlyStopping(monitor='val_loss',patience=20,verbose=3,restore_best_weights=True)
learning_rate=ReduceLROnPlateau(monitor='val_loss',patience=15,verbose=3)
callbacks=[checkpoint,earlystop,learning_rate]

In [24]:
head_model=Flatten()(head_model)
head_model=Dense(64,activation='relu')(head_model)
head_model=Dropout(0.5)(head_model)
head_model=Dense(2,activation='softmax')(head_model)

model=Model(inputs=base_model.input,outputs=head_model)

for layer in base_model.layers:
    layer.trainable=False

In [25]:
import PIL
from PIL import Image

In [26]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelEncoder
batchsize=80
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,epochs=100)

Epoch 1/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9004 - loss: 0.3463
Epoch 1: val_loss improved from inf to 0.24456, saving model to D:\Model\DD_Project\model checkpoint\model.keras
813/813 ━━━━━━━━━━━━━━━━━━━━ 70s 75ms/step - accuracy: 0.9004 - loss: 0.3462 - val_accuracy: 0.9070 - val_loss: 0.2446 - learning_rate: 0.0010
Epoch 2/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9302 - loss: 0.2011
Epoch 2: val_loss did not improve from 0.24456
813/813 ━━━━━━━━━━━━━━━━━━━━ 61s 76ms/step - accuracy: 0.9302 - loss: 0.2011 - val_accuracy: 0.9058 - val_loss: 0.2499 - learning_rate: 0.0010
Epoch 3/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9311 - loss: 0.1925
Epoch 3: val_loss improved from 0.24456 to 0.24204, saving model to D:\Model\DD_Project\model checkpoint\model.keras
813/813 ━━━━━━━━━━━━━━━━━━━━ 60s 74ms/step - accuracy: 0.9311 - loss: 0.1925 - val_accuracy: 0.9002 - val_loss: 0.2420 - learning_rate: 0.0010
Epoch 4/100
813/813 ━━━━━━━━━━

In [27]:
acc_tr,loss_tr=model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

8139/8139 ━━━━━━━━━━━━━━━━━━━━ 727s 89ms/step - accuracy: 0.9433 - loss: 0.1488
0.14631111919879913
0.9442780613899231


In [28]:
acc_vr,loss_vr=model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

2035/2035 ━━━━━━━━━━━━━━━━━━━━ 151s 74ms/step - accuracy: 0.8979 - loss: 0.2661
0.2582075595855713
0.9018186330795288


In [29]:
acc_te,loss_te=model.evaluate(test_data)
print(acc_te)
print(loss_te)

3806/3806 ━━━━━━━━━━━━━━━━━━━━ 308s 81ms/step - accuracy: 0.9486 - loss: 0.1444
0.1514720618724823
0.9463986754417419
